<a href="https://colab.research.google.com/github/pabloderen/BIMnferer/blob/prueba-berna-/Pipe_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import urllib

In [2]:
#We download from de API in Flasgger (http://3.14.88.102:5000/apidocs/) the Pipe list item and it´s parameters
#
jsonfile = urllib.request.urlopen("http://3.14.88.102:5000/api/1/elements/pipes/?limit=6000").read().decode()
j=json.loads(jsonfile)

In [3]:
#Then we make a Data Frame with the json file
df= pd.json_normalize(j)
df.head(3)

,projectId,Category,modelId,_id.$oid,properties_Dimensions_Length,properties_Dimensions_Size,properties_Mechanical_Material,properties_Mechanical_System Classification
0,b.c94f85e5-88fb-402c-bc60-72622f5f85b6,Pipes,d4ea3308-4b3f-d382-cacb-9791f772042b,5eee8651cb573f2fc2216a82,NaN,NaN,NaN,NaN
1,b.c94f85e5-88fb-402c-bc60-72622f5f85b6,Pipes,d4ea3308-4b3f-d382-cacb-9791f772042b,5eee8651cb573f2fc2216a83,NaN,NaN,NaN,NaN
2,b.c94f85e5-88fb-402c-bc60-72622f5f85b6,Pipes,d4ea3308-4b3f-d382-cacb-9791f772042b,5eee8651cb573f2fc2216a84,0.398 ft,"2 1/2""",Copper,Sanitary


In [ ]:
#We will drop the columns that have unuseful information for our case,
#like "projectId", "modelId" and "_id.$oid"
values=["_id.$oid","modelId","projectId"]
df.drop(values, axis=1,inplace=True)

In [7]:
# we can see how many different materials we have, and in some cases are the same, as "CP_PI - Steel - ASTM A53-B" and "Carbon Steel - ASTM A53 B"
df["properties_Mechanical_Material"].unique()

array([nan, 'Copper', 'Steel, Carbon', '<By Category>',
       'Polyvinyl Chloride - Rigid', 'CP_PI - Copper - ASTM B88 H.D.',
       'CP_PI - Steel - ASTM A53-B', 'CP_PI - PVC', 'Cast Iron',
       'Black Steel', 'Carbon Steel', 'Ductile Iron',
       'Polyvinyl Chloride, Rigid', 'Carbon Steel - ASTM A53 B'],
      dtype=object)

In [26]:
# To make such a grouping, we define a function that search in the "properties_Mechanical_Material" and change each row that has another different name
# for the same material
def search(data):
   
    if "A53" in data.upper():
        return "Carbon Steel - ASTM A53 B"
    if "CATEGORY" in data.upper():
        return "NaN"
    if "POLYVINYL" in data.upper():
        return "PVC"
    if "PVC" in data.upper():
        return "PVC"
  
    else:
        return data


In [27]:
# For practical purpuses, we will make a copy of the DataFrame, 
# then apply the function in each row of the "properties_Mechanical_Material" column.
df2=df
df2["properties_Mechanical_Material"]=df2["properties_Mechanical_Material"].apply(lambda x: search(str(x)))


In [28]:
# here is the amount of each material kind
df2["properties_Mechanical_Material"].value_counts()

PVC                               1745
Carbon Steel - ASTM A53 B         1322
Carbon Steel                      1237
Copper                            1102
CP_PI - Copper - ASTM B88 H.D.     424
Steel, Carbon                      121
nan                                 37
Black Steel                          4
NaN                                  4
Cast Iron                            2
Ductile Iron                         2
Name: properties_Mechanical_Material, dtype: int64

In [32]:
df2.groupby(["properties_Mechanical_Material","properties_Mechanical_System Classification"])["Category"].count()

properties_Mechanical_Material  properties_Mechanical_System Classification
Black Steel                     Sanitary                                          4
CP_PI - Copper - ASTM B88 H.D.  Domestic Cold Water                             208
                                Domestic Hot Water                              216
Carbon Steel                    Domestic Cold Water                              34
                                Hydronic Return                                 457
                                Hydronic Supply                                 560
                                Other                                            24
                                Sanitary                                        162
Carbon Steel - ASTM A53 B       Domestic Hot Water                               12
                                Hydronic Return                                 630
                                Hydronic Supply                                 664


In [ ]:
df2["properties_Mechanical_System Classification"].value_counts()

Sanitary               1499
Hydronic Supply        1496
Hydronic Return        1209
Domestic Cold Water     716
Vent                    417
Domestic Hot Water      321
Other                   187
Fire Protection Dry     116
Fire Protection Wet       2
Name: properties_Mechanical_System Classification, dtype: int64

In [33]:
# We drop the rows with properties_Dimensions_Length NaN values.
df3=df2.dropna(axis=0,subset=["properties_Dimensions_Length"])


In [35]:
#Her we start to convert the properties_Dimensions_Length from string to float 
#(we use x.replace(" ft","") and x.replace("  ft",""), notice the spaces in each case)

df3["properties_Dimensions_Length"]= df3["properties_Dimensions_Length"].apply(lambda x: x.replace(" ft",""))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df3["properties_Dimensions_Length"]= df3["properties_Dimensions_Length"].apply(lambda x: float(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#we are analazing pipes, so we know that the dimensions Sizes are the diameter sizes.
#then we can see that because a Size has the diameter simbol, creates another dimension. So 
# we will take of the diameter simbol of all the column.

df3["properties_Dimensions_Size"].value_counts().head(5)

2"     643
4"ø    530
4"     522
6"ø    466
3"     436
Name: properties_Dimensions_Size, dtype: int64

In [ ]:
df3["properties_Dimensions_Size"]= df3["properties_Dimensions_Size"].apply(lambda x: x.replace("ø",""))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Here we eliminate any leading and trailing whitespaces
df3["properties_Dimensions_Size"]= df3["properties_Dimensions_Size"].apply(lambda x: x.strip())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df3["properties_Dimensions_Size"].value_counts()

4"        1070
2"         835
3/4"       727
6"         636
3"         592
1"         580
2 1/2"     316
1 1/4"     257
1/2"       241
1 1/2"     235
16"        146
14"         96
8"          84
12"         64
10"         60
5"          22
1 3/8"       2
Name: properties_Dimensions_Size, dtype: int64

In [ ]:
#The "properties_Mechanical_System Classification" are 9 different types.
df3["properties_Mechanical_System Classification"].nunique()


9

In [ ]:
# Set a number designator to all properties_Mechanical_System Classification
dicc={}
i=0
for i in range(0,9):
  dicc[lista[i]]=i

In [ ]:
#we create a column with this designation
df3["Number_Classification"]=df3["properties_Mechanical_System Classification"].apply(lambda x: dicc[x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#then we get the dummies properties_Dimensions_Size and properties_Mechanical_Material

In [ ]:
df3["properties_Dimensions_Size"].nunique()

17

In [ ]:
material= pd.get_dummies(df3['properties_Mechanical_Material'])


In [ ]:
Dimensions_Size= pd.get_dummies(df3["properties_Dimensions_Size"])


In [ ]:
df4 = pd.concat([df3,material,Dimensions_Size],axis=1)

In [ ]:
#we eliminate some columns that we will not use
droplist=["Category","NaN","properties_Mechanical_System Classification","properties_Mechanical_Material","properties_Dimensions_Size"]
df5=df4.drop(droplist, axis=1)

In [ ]:
df5.head()

,properties_Dimensions_Length,Number_Classification,Copper,PVC,Steel,"1 1/2""","1 1/4""","1 3/8""","1""","1/2""","10""","12""","14""","16""","2 1/2""","2""","3""","3/4""","4""","5""","6""","8"""
2,0.398,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2.536,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0.031,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,0.031,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,0.736,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
#here we define the size of our test size.
from sklearn.model_selection import train_test_split
X = df5.drop('Number_Classification',axis=1)
y = df5['Number_Classification']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [ ]:
#here we train our Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=110)
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=110,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
predictions = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89       445
           1       0.75      0.82      0.78        55
           2       0.78      0.68      0.73       225
           3       0.65      0.64      0.64       118
           4       0.51      0.64      0.57        85
           5       0.83      0.80      0.82       472
           6       0.74      0.78      0.76       354
           7       0.00      0.00      0.00         0
           8       0.77      0.77      0.77        35

    accuracy                           0.79      1789
   macro avg       0.66      0.67      0.66      1789
weighted avg       0.79      0.79      0.79      1789



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_test,predictions))

[[399   1   5  40   0   0   0   0   0]
 [  1  45   5   0   0   2   2   0   0]
 [  4  11 153   0  38  11   8   0   0]
 [ 42   0   1  75   0   0   0   0   0]
 [  0   1  24   0  54   4   2   0   0]
 [  2   0   4   0   3 379  78   2   4]
 [  0   2   3   0  10  60 275   0   4]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   2   6   0  27]]
